In [92]:
import pandas as pd
import pulp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import random
from pulp import PULP_CBC_CMD

In [93]:
#define the function for users to enter the age period, number of male and female students and number of menu to generate
def age_period():
    while True:
        print('Please select students\' age period:')
        print('A: 4-6 years old')
        print('B: 7-10 years old')
        print('C: 11-14 years old')
        print('D: 15-18 years old')
        age_period = input('Enter A, B, C or D:')
        if age_period.upper() in ['A','B','C','D']:
            return age_period
        else:
            print('Invalid input. Please try again.')
def count_students():
    while True:
        print('Please enter the number of male student')
        male_students = input()
        if male_students.isdigit():
            break
        else:
            print('Invalid input. Please input an integer.')
    while True:
        print('Please enter the number of female student')
        female_students = input()
        if female_students.isdigit():
            break
        else:
            print('Invalid input. Please input an integer.')
    return int(male_students), int(female_students)
def number_of_menu_to_generate():
    while True:
        n = input('Please enter the number of menu to generate:')
        if n.isdigit():
            break
        else:
            print('Invalid input. Please input an integer.')

In [94]:
#let users to enter, the parameters are fixed for now
#tag = age_period()
#malenum, femalenum = count_students()
#n = number_of_menu_to_generate()
tag = 'D'
malenum, femalenum = 500, 500
n = 5

In [95]:
#set up unpreferred food list such as allergy food
unpreferred_food = ['Quinoa','Peanut','Zucchini','raw']
#set up unpreperred food list such as baby formula
unpreferred_group = ['BF','BFD','BFG','BFJ','BFP']

In [96]:
#read the government dietary recommendation file
sheet_dict = {'A':0, 'B':1, 'C':2, 'D':3}
diet_recommendation = pd.read_excel('government_dietry_reccomandation.xlsx', sheet_name=sheet_dict[tag] , index_col=0)
#sum up the toatal demand of each nutrient based on the number of male and female students
diet_recommendation['demand'] = diet_recommendation['Males']*malenum + diet_recommendation['Females']*femalenum
#the meal takes up 40% of total demand of each nutrient within a day
diet_recommendation['demand'] = diet_recommendation['demand']*0.4

The part below is to cleanse the menu data. It just shows how we process the data from the internet. The users should have a menu of their own, so they can just upload their own menu and skip next 4 blocks.

In [109]:
#read the McCane and Widdowsons Food Composition Database
food_composition = pd.read_excel('McCance_Widdowsons_Composition_of_Foods_Integrated_Dataset_2021.xlsx', index_col=0, sheet_name='1.3 Proximates')
columns_in_need = ['Food Name','Description','Group', 'Protein (g)','Carbohydrate (g)',
                   'Energy (kcal) (kcal)','Total sugars (g)','Satd FA /100g fd (g)',
                   'Mono FA /100g food (g)','Poly FA /100g food (g)']
food_composition = food_composition[columns_in_need]
food_composition.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2889 entries, nan to 11-1146
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Food Name               2887 non-null   object
 1   Description             2887 non-null   object
 2   Group                   2887 non-null   object
 3   Protein (g)             2889 non-null   object
 4   Carbohydrate (g)        2887 non-null   object
 5   Energy (kcal) (kcal)    2888 non-null   object
 6   Total sugars (g)        2887 non-null   object
 7   Satd FA /100g fd (g)    2875 non-null   object
 8   Mono FA /100g food (g)  2874 non-null   object
 9   Poly FA /100g food (g)  2873 non-null   object
dtypes: object(10)
memory usage: 248.3+ KB


In [98]:
#read the sodium data in another sheet
food_composition2 = pd.read_excel('McCance_Widdowsons_Composition_of_Foods_Integrated_Dataset_2021.xlsx', index_col=0, sheet_name='1.4 Inorganics')
food_composition2 = food_composition2['Sodium (mg)']
#concat the two dataframe
nutrient = pd.concat([food_composition, food_composition2], axis=1)
#drop 2 rows of redundant rows
nutrient = nutrient.iloc[2:]
#replcae the 'Tr' and 'N' to 0
nutrient.replace('Tr',0, inplace=True)
nutrient.replace('N',0, inplace=True)
#drop the alcoholic group
nutrient = nutrient[~nutrient['Group'].str.startswith('Q')]

In [99]:
#The amount of menu is too large and contains too many similar menu
#create a tf-idf vectorizer
food_type = TfidfVectorizer().fit_transform(nutrient['Food Name'])

# split 400 kinds of different food
type_num = 400

# conduct clustering
kmeans = KMeans(n_clusters=type_num, random_state=0)
kmeans.fit(food_type)

# attach the final result to the dataframe
df = pd.DataFrame({'Dish': nutrient['Food Name'], 'Cluster': kmeans.labels_})
nutrient = pd.concat([nutrient, df['Cluster']], axis=1)
#显示结果

In [100]:
#each cluster retain one sample
nutrient = nutrient.reset_index()
nutrient = nutrient.groupby('Cluster',as_index=False).head(1)
nutrient = nutrient.sort_values('Food Name', ascending=True)   
nutrient = nutrient.set_index('index')
#save the final result
nutrient.to_excel('different_food.xlsx')

This is the end of data cleansing. The users can skip the 4 blocks above and upload their own menu below. After the different_food.xlsx is saved. We priced the food manually based on the price of the food material which become our different_food_with_price.xlsx file.

In [101]:
#read the menu with price file
nutrientprice = pd.read_excel('different_food_with_price.xlsx', index_col=0)
#the nutrientprice may be edited, so we have the displaytable as fixed database
displaytable = nutrientprice

In [102]:
#The nutrient is calculated based on 100g food, but the calculation is based on 10g, so we divide 10
number_col = nutrientprice.select_dtypes(include=[int,float]).columns
nutrientprice[number_col] = nutrientprice[number_col].apply(lambda x: x/10)
#remove the food group that is not needed
nutrientprice = nutrientprice[nutrientprice['Group'].str.startswith(('A','J','M','D','F','P'))]
#The sodium nutrient dimension is mg, the sodium demand dimension is g, so we divide 100
nutrientprice.loc[:, 'Salt'] = nutrientprice['Salt'] / 100
#replace the nan value with 0
nutrientprice = nutrientprice.fillna(0)

In [103]:
#delet the unpreferred food and group
for i in unpreferred_food:
    nutrient = nutrientprice[~nutrientprice['Food Name'].str.contains(i, case = False)]
for i in unpreferred_group:
    nutrientprice = nutrientprice[nutrientprice['Group'] != i]

In [104]:
#the total demand of each nutrient
total_demand = diet_recommendation['demand']

In [105]:
def find_solution():
    ## Declare the global variable nutrientprice to be used or modified in this scope
    global nutrientprice
    #set a number big enough to be used later
    M = 100000*(malenum+femalenum)
    #get all the food index
    food_index = list(nutrientprice.index)
    #creat an empty dictionary to store the solution
    solution_dic = {}
    #create the problem
    prob = pulp.LpProblem("Food_Selection", pulp.LpMinimize)
    #create the decision variables
    pick_routes = pulp.LpVariable.dicts("Pickroutes", (food_index), lowBound=0, cat=pulp.LpInteger)
    #create the binary variables to identity whether a certain food is selected
    binary_routes = pulp.LpVariable.dicts("Binary_routes", (food_index), lowBound=0, upBound=1, cat=pulp.LpBinary)
    #set the objective function
    prob += pulp.lpSum([nutrientprice.loc[i,'price']*pick_routes[i] for i in food_index]),'Objective Function'
    #when a certain food is selected, set binary_routes to 1 respectively
    for i in food_index:
        prob += pick_routes[i] <= M * binary_routes[i]
        prob += pick_routes[i] >= binary_routes[i]
                
    #Ensure that there is one and only one type of each kind of ingredient.
    type_list=[list(nutrientprice[nutrientprice['Group'].str.startswith('A')].index),
    list(nutrientprice[nutrientprice['Group'].str.startswith(('J','M'))].index),
    list(nutrientprice[nutrientprice['Group'].str.startswith('D')].index),
    list(nutrientprice[nutrientprice['Group'].str.startswith('F')].index),
    list(nutrientprice[nutrientprice['Group'].str.startswith(('P','B'))].index)]
    for i in range(len(type_list)):
        prob += pulp.lpSum([binary_routes[j] for j in type_list[i]]) == 1
        
    #set a maximum amount of some group(This part can be edited based on certain cirmustance)
    for i in list(nutrientprice[nutrientprice['Group'].str.startswith('D')].index):
        prob += pick_routes[i] <= 9*(malenum+femalenum)
    for i in list(nutrientprice[nutrientprice['Group'].str.startswith('A')].index):
        prob += pick_routes[i] <= 9*(malenum+femalenum)
    for i in list(nutrientprice[nutrientprice['Group'].str.startswith(('J','M'))].index):
        prob += pick_routes[i] <= 8*(malenum+femalenum)
    for i in list(nutrientprice[nutrientprice['Group'].str.startswith('D')].index):
        prob += pick_routes[i] <= 9*(malenum+femalenum)
    for i in list(nutrientprice[nutrientprice['Group'].str.startswith('A')].index):
        prob += pick_routes[i] <= 9*(malenum+femalenum)


    #ensuere the demand of each nutrient is met
    prob += pulp.lpSum([nutrientprice.loc[j,'Enenergy(kcak/day)']*pick_routes[j] for j in food_index]) >= total_demand['Enenergy(kcak/day)']
    prob += pulp.lpSum([nutrientprice.loc[j,'Protein']*pick_routes[j] for j in food_index]) >= total_demand['Protein']
    prob += pulp.lpSum([nutrientprice.loc[j,'Carbohydrate']*pick_routes[j] for j in food_index]) >= total_demand['Carbohydrate']
    prob += pulp.lpSum([nutrientprice.loc[j,'Free Sugar']*pick_routes[j] for j in food_index]) <= total_demand['Free Sugar']
    prob += pulp.lpSum([nutrientprice.loc[j,'Salt']*pick_routes[j] for j in food_index]) <= total_demand['Salt']
    prob += pulp.lpSum([nutrientprice.loc[j,'Saturated fat']*pick_routes[j] for j in food_index]) <= total_demand['Saturated fat']
    prob += pulp.lpSum([nutrientprice.loc[j,'Monounsaturated fat']*pick_routes[j] for j in food_index]) >= total_demand['Monounsaturated fat']
    prob += pulp.lpSum([nutrientprice.loc[j,'Polyunsaturated fat']*pick_routes[j] for j in food_index]) >= total_demand['Polyunsaturated fat']
    
    #slove the problem
    prob.solve(PULP_CBC_CMD(msg=False))
    #check whether the problem is solved
    if prob.status != 1:
        return False, False
    #get the result
    for v in prob.variables():
        #only analyse the LpInteger variables with value larger than 0
        if v.varValue != 0 and v.name.split('_')[0] == 'Pickroutes':#None
            try:
                #get the food index
                chosen = v.name.split('_')[1:3]
                chosenindex = '-'.join(chosen)
                #find the food name based on food index
                fn = nutrientprice.loc[chosenindex,'Food Name']
                #add the result to the dictionary(the value is timed 10 to match the dimension)
                solution_dic.update({fn: v.varValue*10}) 
                #To ensure the variety of food, drop this certain food group 
                chosen_group = nutrientprice.loc[chosenindex,'Group']
                nutrientprice = nutrientprice[nutrientprice['Group'] != chosen_group]
            except:
                pass
    return solution_dic, prob.objective.value()

In [106]:
#generate certain amount of value
result = []
cost = []
for i in range(n):
    out, value = find_solution()
    #identify whether there is a proper solution
    if out == False:
        #if there is no proper solution, stop the loop and hint the users
        print(f'So far, {i} days of receipe has been generated')
        print('But no more receipe can be generated, please adjust the menu or objectives')
        break
    #if there is a proper solution, add it to result
    else:
        result.append(out)
        cost.append(value)
# #randomly shuffle the result
# combine = list(zip(result,cost))
# random.shuffle(combine)
# result,cost = zip(*combine)
def print_menu(dct):
    # calculate the width of each column
    key_width = max(len(str(key)) for key in dct.keys()) + 3  
    value_width = max(max(len(str(value)) for value in dct.values()), len('Amount(g)/(ml)'))    
    
    # print the headers
    print(f"{'Course Name':<{key_width}}{'Amount(g)/(ml)':<{value_width}}")
    print('-' * (key_width + value_width))  
    
    # print every key and value
    for key, value in dct.items():
        print(f"{key:<{key_width}}{value:<{value_width}}")
#print the result
d =1
for i in range(len(result)):
    print(f'Day {d}') 
    print_menu(result[i])
    print('\n')
    print(f'The total cost of this menu is {cost[i]}p')
    print('\n')
    d += 1  

So far, 3 days of receipe has been generated
But no more receipe can be generated, please adjust the menu or objectives
Day 1
Course Name                                                Amount(g)/(ml)
-------------------------------------------------------------------------
Breakfast cereal, instant hot oat, plain, raw, fortified   90000.0       
Tamarind                                                   58630.0       
Apple juice concentrate, unsweetened, commerical           16140.0       
Nut croquettes, fried in vegetable oil, homemade           90000.0       
Beef, fat, average, raw, extra trimmed                     13560.0       


The total cost of this menu is 1174.283p


Day 2
Course Name                                  Amount(g)/(ml)
-----------------------------------------------------------
Porridge oats, unfortified                   90000.0       
Beans, pigeon peas, dahl, dried, raw         90000.0       
Blackberry and apple, stewed without sugar   41880.0       
Milk 